In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# グラフのスタイルを指定
plt.style.use('seaborn-darkgrid')

In [21]:
last = False
num_epochs = 3

In [22]:
def fix_seed(seed=0):
    # numpy
    np.random.seed(seed)
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

fix_seed()

In [23]:
from HandGesture_module_v2 import ai as gesture

model_name='resnet'
num_classes=10
feature_extract=False

model_ft, input_size = gesture.initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, binary=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_ft = model_ft.to(device)

In [24]:
trans = transforms.Compose([transforms.ToTensor(),transforms.Resize(224)])
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=trans,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           transform=trans,
                                           download=True)
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)
dataloaders_dict = {'train': train_loader, 'val': test_loader}
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [25]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

# 最適化手法を設定
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [26]:
# Train and evaluate
model_ft, train_acc_hist_tensor, val_acc_hist_tensor = gesture.train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, last, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/2
----------
train Loss: 0.0952 Acc: 0.9764
val Loss: 0.0205 Acc: 0.9936

Epoch 1/2
----------
train Loss: 0.0230 Acc: 0.9937
val Loss: 0.0158 Acc: 0.9949

Epoch 2/2
----------
train Loss: 0.0145 Acc: 0.9958
val Loss: 0.0136 Acc: 0.9959

Training complete in 7m 41s
Best val Acc: 0.995900


In [27]:
print(model_ft)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
import os

In [29]:
out_dir='.'

In [30]:
#torch.save(model_ft.state_dict(), os.path.join(out_dir,'model_weights.pth'))

In [31]:
#torch.save(model_ft.state_dict(), os.path.join(out_dir,'model_mnist.pt'))

In [32]:
# LibTorch形式でモデルをエクスポートします

#バッチサイズ32　チャンネル1　入力サイズ224x224
input_tensor = torch.randn(64,1, 224, 224)
# input_tensor = input_tensor.unsqueeze(0)
input_tensor = input_tensor.cuda()

#traced_script_module = torch.jit.trace(model_ft, input_tensor)
#traced_script_module.save('model_weights_c++.ptc')
#traced_script_module.save('model_weights_c++.pt')

In [33]:

if torch.cuda.is_available():
    device = torch.device("cuda")          # CUDAデバイスオブジェクトを作成
    print("CUDAが使用可能です")
    print("現在のCUDAデバイス：", torch.cuda.get_device_name())
else:
    device = torch.device("cpu")
    print("CUDAは使用できません")


print("PyTorchバージョン：", torch.__version__)
if torch.cuda.is_available():
    print("CUDAバージョン：", torch.version.cuda)
else:
    print("CUDAは使用できません")

CUDAが使用可能です
現在のCUDAデバイス： NVIDIA GeForce RTX 3070 Ti
PyTorchバージョン： 1.11.0
CUDAバージョン： 11.3


In [34]:
print(torch.__version__)

1.11.0


In [35]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")          # CUDAデバイスオブジェクトを作成
    print("CUDAが使用可能です")
    print("現在のCUDAデバイス：", torch.cuda.get_device_name())
else:
    device = torch.device("cpu")
    print("CUDAは使用できません")


print("PyTorchバージョン：", torch.__version__)
if torch.cuda.is_available():
    print("CUDAバージョン：", torch.version.cuda)
else:
    print("CUDAは使用できません")

print(torch.__version__)

CUDAが使用可能です
現在のCUDAデバイス： NVIDIA GeForce RTX 3070 Ti
PyTorchバージョン： 1.11.0
CUDAバージョン： 11.3
1.11.0


: 